In [ ]:
!pip install torch transformers accelerate bitsandbytes datasets jsonlines


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 📌 Nombre del modelo en Hugging Face
model_name = "meta-llama/Meta-Llama-3-8B"

# 🔄 Cargar el tokenizador y modelo en GPU con `bitsandbytes` (para menor consumo de memoria)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    device_map="auto",  # Mapea el modelo a la GPU disponible
    torch_dtype="auto"  # Usa la mejor precisión posible
)

print("✅ Llama 3.1 8B cargado correctamente.")


ModuleNotFoundError: No module named 'transformers'

In [ ]:
from datasets import load_dataset

# 📂 Cargar el dataset JSONL
dataset = load_dataset("json", data_files="contratos_dataset.jsonl", split="train")

# 🔄 Tokenizar los datos
def tokenize_function(example):
    return tokenizer(example["prompt"], text_target=example["response"], padding="max_length", truncation=True)

# 🏷 Aplicar tokenización
dataset = dataset.map(tokenize_function, batched=True)


In [ ]:
from transformers import TrainingArguments, Trainer

# 📂 Configurar la carpeta donde se guardarán los checkpoints
training_args = TrainingArguments(
    output_dir="./llama3_contratos",
    per_device_train_batch_size=2,  # Ajusta según la VRAM de tu GPU
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,
    fp16=True,  # Usa FP16 para optimizar memoria
    report_to="none"
)

# 📌 Configurar el trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# 🚀 Iniciar entrenamiento
trainer.train()


In [ ]:
# 💾 Guardar el modelo ajustado
model.save_pretrained("llama3_contratos_tuned")
tokenizer.save_pretrained("llama3_contratos_tuned")

print("✅ Modelo guardado en 'llama3_contratos_tuned'.")


In [ ]:
# 📂 Cargar el modelo ajustado
model_path = "llama3_contratos_tuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

# 🔍 Ejemplo de contrato nuevo
prompt = "Analiza este contrato y determina si cumple con la ley dominicana:\nEl inquilino renuncia a cualquier reclamo legal."
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# 🚀 Generar respuesta
outputs = model.generate(**inputs, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
